In [1]:
import sys
import numpy as np
sys.path.insert(0,'../')
sys.path.insert(0,'../goflow')
sys.path.insert(0,'../..')
sys.path.insert(0,'../../kirchhoff-hydro')
sys.path.insert(0,'../../kirchhoff-circuits')
import goflow.models as gfm
import goflow.init_ivp as gi

import kirchhoff.circuit_flow as kfc
import hailhydro.flow_init as hf

# #initialize circuit+flow pattern for a dynamic solver system 'morpheus', set circuit_parameters cp
cp={
    'skeleton': kfc.init_crystal.init_graph_from_crystal('hexagonal',3),
    'source':'dipole_border',
    'plexus':'default',
}
circuit=kfc.setup_flow_circuit(cp)
flow=hf.initialize_flow_on_circuit(circuit)

circuit(): initialized and ready for (some) action :)
set_source_landscape():  is set and consistent :)
set_plexus_landscape(): default is set and consistent :)


In [4]:
system_parameters={
    't0': 0.,
    't1': 1.,
    'x0': np.power(flow.circuit.edges['conductivity']/flow.circuit.scales['conductance'],0.25),
    'alpha_0': 10.,
    'alpha_1': 1.
}

solver_options={
    't_eval':np.linspace(system_parameters['t0'],system_parameters['t1'],num=300),
    'events':'flatlining',
}

morpheus=gfm.init_custom_adaptation(flow,sys_pars=system_parameters,solv_opt=solver_options)
nsol=morpheus.nsolve()
morpheus.jac=False
cost=[murray.calc_cost_stimuli(t,y,*morpheus.solver_options['args']) for t,y in zip(nsol.t,nsol.y.transpose())]

TypeError: 'str' object is not callable

In [ ]:


fig,axs=plt.subplots(2,1,figsize=(12,6),sharex=True)
axs[0].plot(nsol.t,nsol.y.transpose(),alpha=0.2,color='b')
axs[1].plot(nsol.t,cost,alpha=0.1,color='b')

for i in range(2):
    axs[i].grid(True)
# axs[0].set_yscale('log')  
axs[1].set_xlabel(r'time $t$')
axs[0].set_ylabel(r'radii $r$')
axs[1].set_ylabel(r'metabolic cost $\Gamma$')
plt.show()

In [ ]:
flow.circuit.edges['conductivity']=nsol.y.transpose()[-1]
flow.circuit.edges['flow_rate']=flow.calc_configuration_flow()
flow.circuit.draw_weight_scaling=5.
fig=flow.circuit.plot_circuit({'width':'flow_rate'})
fig.show()